<a href="https://colab.research.google.com/github/utkarshg1/Pytorch/blob/main/Pytorch_Multiclass_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data creation

In [14]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [15]:
# Dataset parameters
N_SAMPLES = 4000
N_FEATURES = 10
N_INFORMATIVE = 5
N_CLASSES = 4
RANDOM_STATE = 42

# Make classification
X, y = make_classification(n_samples=N_SAMPLES, n_features=N_FEATURES,
                           n_informative=N_INFORMATIVE, n_classes=N_CLASSES,
                           random_state=RANDOM_STATE)

In [16]:
X

array([[ 0.81203507,  2.25275853,  0.76499235, ...,  0.48318329,
         1.52099965, -1.83205977],
       [ 0.20196614,  0.0830052 , -1.28840013, ..., -0.97458052,
         0.23646668,  1.05807438],
       [ 0.849731  ,  0.28688947, -0.60478079, ..., -0.91572863,
         0.69042442, -0.50842179],
       ...,
       [-0.50696395,  3.40808192, -0.88127988, ...,  0.56784662,
        -0.56586092, -5.09895611],
       [ 0.40467275, -1.21416881,  0.42682712, ..., -0.92663425,
         0.87655389, -0.90506585],
       [ 1.17814108,  0.68888479, -1.11387012, ...,  0.32892542,
        -0.1104735 ,  1.63263462]])

In [17]:
y

array([3, 1, 3, ..., 1, 2, 3])

### Convert numpy to tensor

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [19]:
X = torch.from_numpy(X).type(torch.float32).to(device)
y = torch.from_numpy(y).type(torch.float32).to(device)

In [20]:
X.dtype

torch.float32

In [21]:
y.dtype

torch.float32

### Train Test split

In [25]:
xtrain, xtest, ytrain, test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
xtrain.shape

torch.Size([2800, 10])

In [27]:
xtest.shape

torch.Size([1200, 10])